In [1]:
import pandas as pd
import regex as re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.tokenize import word_tokenize
import nltk
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('punkt')
pd.set_option('display.max_rows',None)
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /Users/ranggi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Data MUA for Memory Based System Recommendation

In [2]:
class ContentBasedFiltering:
    def case_folding(self, text):
        if type(text) == str:
            text = text.lower()
        return text

    def punctuation_removal(self, text):
        if type(text) == str:
            text = re.sub(r'[^\w\s]', '', text)
        return text

    def tokenize(self, text):
        tokens = word_tokenize(text)
        return tokens
        
    def stopwords_removal(self, text):
        stopword_factory = StopWordRemoverFactory()
        stopword_remover = stopword_factory.create_stop_word_remover()
        no_stopword = stopword_remover.remove(text)
        return no_stopword

    def preprocessing(self, text):
        text = self.case_folding(text)
        text = self.punctuation_removal(text)
        text = self.stopwords_removal(text)
        return text

In [3]:
data = pd.read_excel('./web_mua/static/Data MUA 2.xlsx')
data.head()

,no,nama,nama_MUA,latitude,longitude,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating
0,1,Anggia Pradita A,Intan Rias Pengantin,-8.233613,113.686384,la tulipe cover foundation,natural (putih),light,cool,Murah,-8.233613,113.66733,4
1,1,Anggia Pradita A,Intan Rias Pengantin,-8.233613,113.686384,la tulipe cover foundation,suntan (sawo matang),medium,warm,Murah,-8.233613,113.66733,4
2,2,VIKA APRILIA SINTA PUTRI,Sanggar Rias Cantik Tina,-8.221109,113.680964,la tulipe cover foundation,rachel 1 (kuning langsat),medium,neutral,Murah,-8.233613,113.66733,5
3,2,VIKA APRILIA SINTA PUTRI,Sanggar Rias Cantik Tina,-8.221109,113.680964,la tulipe cover foundation,suntan (sawo matang),medium,warm,Murah,-8.233613,113.66733,5
4,3,septia maharani,Rias Pengantin Azizah,-8.234863,113.680670,la tulipe cover foundation,natural (putih),light,cool,Sedang,-8.233613,113.66733,3


In [4]:
def cleaning_data(data):
    cleaned_data = []
    CBF = ContentBasedFiltering()
    for i in range(len(data)):
        cleaned_data.append(CBF.preprocessing(str(data[i])))
    return cleaned_data

In [5]:
#praproses data untuk nama
data['nama'] = cleaning_data(data['nama'])

In [6]:
data_for_model = data.copy()
data_for_user = data.copy()

#praproses data untuk deskripsi
data_for_model['desc'] = data['produk_makeup'] + ' ' + data['shade'] + ' ' + data['skin_color'] + ' ' + data['skin_undertone']
data_for_model['desc'] = cleaning_data(data_for_model['desc'])
data_for_user['desc'] = data['produk_makeup'] + ' ' + data['skin_color'] + ' ' + data['skin_undertone']
data_for_user['desc'] = cleaning_data(data_for_user['desc'])

In [7]:
# data_for_model = data_for_model.groupby('no').agg({'nama':'first', 'nama_MUA':'first', 'kategori_harga': 'first', 'latitude':'first', 'longtitude':'first', 'rating':'first','desc':' '.join}).reset_index()
data_for_model = data_for_model.sort_values(by=['nama'], ignore_index=True)

data_for_user = data_for_user.sort_values(by=['nama'], ignore_index=True)
data_for_user = data_for_user.drop(columns=['shade'])

In [8]:
data_for_model.head()

,no,nama,nama_MUA,latitude,longitude,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating,desc
0,53,adera zia,Dinda Wijaya Make Up,-8.193711,113.619405,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,Sedang,-8.193711,113.616830,5,lt pro smooth corrector cream foundation natur...
1,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,Sedang,-8.139019,113.728548,4,lt pro smooth corrector cream foundation natur...
2,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,la tulipe cover foundation,suntan (sawo matang),medium,warm,Sedang,-8.139019,113.728548,4,la tulipe cover foundation suntan sawo matang ...
3,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,la tulipe cover foundation,suntan (sawo matang),medium,warm,Sedang,-8.157255,113.652074,3,la tulipe cover foundation suntan sawo matang ...
4,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,LT Pro smooth corrector cream foundation,plum rose (putih),light,cool,Sedang,-8.157255,113.652074,3,lt pro smooth corrector cream foundation plum ...


In [9]:
data_for_user.head()

,no,nama,nama_MUA,latitude,longitude,produk_makeup,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating,desc
0,53,adera zia,Dinda Wijaya Make Up,-8.193711,113.619405,LT Pro smooth corrector cream foundation,medium,neutral,Sedang,-8.193711,113.616830,5,lt pro smooth corrector cream foundation mediu...
1,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,LT Pro smooth corrector cream foundation,medium,neutral,Sedang,-8.139019,113.728548,4,lt pro smooth corrector cream foundation mediu...
2,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,la tulipe cover foundation,medium,warm,Sedang,-8.139019,113.728548,4,la tulipe cover foundation medium warm
3,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,la tulipe cover foundation,medium,warm,Sedang,-8.157255,113.652074,3,la tulipe cover foundation medium warm
4,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,LT Pro smooth corrector cream foundation,light,cool,Sedang,-8.157255,113.652074,3,lt pro smooth corrector cream foundation light...


In [10]:
data_for_model.shape

(179, 14)

In [11]:
from sklearn.preprocessing import LabelEncoder

harga_labels = LabelEncoder()
harga_labels = harga_labels.fit(data['kategori_harga'])
harga_labels.classes_ = np.array(['Murah', 'Sedang', 'Mahal'])
data_for_model['kategori_harga'] = harga_labels.transform(data_for_model['kategori_harga'])
data_for_user['kategori_harga'] = harga_labels.transform(data_for_user['kategori_harga'])

In [12]:
data_for_model.head()

,no,nama,nama_MUA,latitude,longitude,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating,desc
0,53,adera zia,Dinda Wijaya Make Up,-8.193711,113.619405,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,1,-8.193711,113.616830,5,lt pro smooth corrector cream foundation natur...
1,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,1,-8.139019,113.728548,4,lt pro smooth corrector cream foundation natur...
2,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,la tulipe cover foundation,suntan (sawo matang),medium,warm,1,-8.139019,113.728548,4,la tulipe cover foundation suntan sawo matang ...
3,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,la tulipe cover foundation,suntan (sawo matang),medium,warm,1,-8.157255,113.652074,3,la tulipe cover foundation suntan sawo matang ...
4,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,LT Pro smooth corrector cream foundation,plum rose (putih),light,cool,1,-8.157255,113.652074,3,lt pro smooth corrector cream foundation plum ...


In [13]:
data_for_user.head()

,no,nama,nama_MUA,latitude,longitude,produk_makeup,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating,desc
0,53,adera zia,Dinda Wijaya Make Up,-8.193711,113.619405,LT Pro smooth corrector cream foundation,medium,neutral,1,-8.193711,113.616830,5,lt pro smooth corrector cream foundation mediu...
1,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,LT Pro smooth corrector cream foundation,medium,neutral,1,-8.139019,113.728548,4,lt pro smooth corrector cream foundation mediu...
2,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,la tulipe cover foundation,medium,warm,1,-8.139019,113.728548,4,la tulipe cover foundation medium warm
3,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,la tulipe cover foundation,medium,warm,1,-8.157255,113.652074,3,la tulipe cover foundation medium warm
4,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,LT Pro smooth corrector cream foundation,light,cool,1,-8.157255,113.652074,3,lt pro smooth corrector cream foundation light...


## User Input

In [14]:
# use this parameter to get some MUA recommendations
def data_preparation(data_for_model, data_for_user, index_user, user):
    du = data_for_user.copy()
    query_data = du.iloc[[index_user]].reset_index()
    query_data_todict = query_data.to_dict('index')[0]
    part_of_model = data_for_model.copy()
    part_of_model = part_of_model.loc[part_of_model.nama_MUA != query_data.nama_MUA[0]].reset_index()
    part_of_query = part_of_model.loc[part_of_model['nama'] == user].reset_index()
    part_of_model = part_of_model.reset_index()
    part_of_model = part_of_model.drop(columns='index')
    return query_data_todict, part_of_query, part_of_model

# example of input data
find_user = data_for_user.index[data_for_user['nama'] == 'adera zia'].to_list()
txt_user = 'adera zia'
query_data_todict, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, find_user[0], txt_user)
# part_of_model = data_preparation(data_for_model, data_for_user, find_user[0], txt_user, first=False)

## Calculating Cosine Similarity for Content Based Model

In [93]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import NMF
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim


def doc2vec_vector(part_of_model, query_data_todict):
    copy_md = part_of_model.copy()
    md = pd.DataFrame(copy_md["desc"])
    md.loc[len(md)] = query_data_todict["desc"]
    words_split = []
    for i in range(len(md)):
        words_split.append([word for word in md["desc"][i].split()])
    documents = []
    for i, list_of_words in enumerate(words_split):
        documents.append(gensim.models.doc2vec.TaggedDocument(words=list_of_words, tags=['doc'+str(i)]))
    model = gensim.models.doc2vec.Doc2Vec(vector_size=40, min_count=2, epochs=30)
    model.build_vocab(documents)
    model.train(documents, total_examples=model.corpus_count, epochs=30)
    # vectorizer = TfidfVectorizer()
    # X = vectorizer.fit_transform(md["desc"])
    # nmf_model = NMF(10)
    # doc_topic = nmf_model.fit_transform(X)
    vec_model = []
    for i in range((len(part_of_model))):
        vec_model.append(model.infer_vector(md["desc"][i].split()))
    vec_query = model.infer_vector(md["desc"][len(md["desc"])-1].split())
    # print(vec_query)
    return vec_model, vec_query

In [94]:
from geopy.distance import great_circle
import math
def calculate_distance(query_data_todict, part_of_model):
        loc1 = (query_data_todict['latitude'], query_data_todict['longtitude'])
        distance_loc = []
        for i in range(len(part_of_model.index)):
            loc2 = (part_of_model['latitude'][i], part_of_model['longtitude'][i])
            tmp = great_circle(loc1, loc2).km
            value = math.ceil(tmp * 100) / 100
            distance_loc.append(value)
        return distance_loc

In [95]:
from gensim import similarities
from scipy import spatial 
import torch

def doc2vec_model(sentences1, sentences2, model):
    res = []
    for sentence1, sentence2 in zip(sentences1, sentences2):
        res.append(model.similarity_unseen_docs(sentence1, sentence2))
    return torch.tensor(res)

def get_numtext_content(query_data_todict, part_of_model):
    vec_model, vec_query = doc2vec_vector(part_of_model, query_data_todict)
    cos_sim = []
    for i in range(len(vec_model)):
        cos_sim.append(spatial.distance.cosine(vec_model[i], vec_query))
    # cos_sim = doc2vec_model(part_of_model["desc"].values.tolist(), part_of_model["desc"].values.tolist(), model)
    # data_sim = cos_sim.reshape(-1,1)
    distance = calculate_distance(query_data_todict, part_of_model)
    return cos_sim, distance

In [96]:
x, y = get_numtext_content(query_data_todict, part_of_model)

In [342]:
from sklearn.preprocessing import StandardScaler

def get_values(part_of_model):
    tmp_data = part_of_model[['kategori_harga','sim', 'latitude', 'longitude']].to_numpy()
    sim_num = cosine_similarity(tmp_data)
    sim_num = np.array([np.mean(p) for p in sim_num])
    return sim_num

In [480]:
def value_of_mua(query_data_todict, part_of_model):
    sim, distance = get_numtext_content(query_data_todict, part_of_model)
    part_of_model["sim"] = sim
    part_of_model["distance"] = distance
    sim_num = get_values(part_of_model)
    part_of_model["sim_num"] = sim_num
    part_of_model = part_of_model.sort_values(by=['nama_MUA'])
    nama_mua = part_of_model["nama_MUA"].unique()
    value_mua = np.empty(len(nama_mua), dtype=float)
    for i in range(len(nama_mua)):
        tmp_val = []
        for j in range(len(part_of_model.index)):
            if nama_mua[i] == part_of_model.nama_MUA[j]:
                tmp_val.append(part_of_model["sim_num"][j])
        value_mua[i] = np.mean(tmp_val)
    part_of_model = part_of_model.reset_index()
    part_of_model = part_of_model.drop(columns='index')
    return nama_mua, value_mua, part_of_model

## FIND USER

In [481]:
unique_user = data_for_model['nama'].unique()
unique_user
query_user = ['adera zia', 'anggia pradita a', 'anissa serina', 'calista devi aqilah', 'callista febriana erwanto', 'dwi rahma', 'maulina vira', 
              'vika aprilia sinta putri', 'yuricha mahargiyani wijaya', 'zainata hanum masruroh', 'zyanti laila']
selected_user = []
for i in query_user:
    find_user = data_for_model.index[data_for_model['nama'] == i].to_list()
    selected_user.append(find_user[0])
selected_user

[0, 10, 20, 33, 45, 73, 101, 142, 151, 161, 169]

## HYBRID FILTERING

In [482]:
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import preprocessing as pre
import statistics 

In [598]:
#chosen user for testing
def hybrid_evaluation(data_for_model, data_for_user, selected_user, query_user):
    actual_hyb = []
    table_hyb = []
    rating_hyb = []
    for index_user in range(len(selected_user)):
        query_data_todict, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[index_user], query_user[index_user])
        nama_mua, value_mua, part_of_model = value_of_mua(query_data_todict, part_of_model)
        pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        col_model = pivot.columns.tolist()
        val_model = np.array(pivot.values.tolist(), dtype=float)
        nama_user = np.array(pivot.index.tolist())
        pivot_query = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_query = pivot_query.values.tolist()
        col_query = pivot_query.columns.tolist()
        feature = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_query)):
                if col_model[i] == col_query[j]:
                    feature[i] = val_query[0][j]
                    break
        
        nmf_model = NMF(3)
        f = nmf_model.fit_transform([feature])
        m = nmf_model.fit_transform(val_model)
        cos_sim_user = cosine_similarity(f, m)

        weight_user = np.empty(len(part_of_model), dtype=float)
        for i in range(len(part_of_model)):
            for j in range(len(nama_user)):
                if part_of_model["nama"][i] == nama_user[j]:
                    weight_user[i] = cos_sim_user[0][j]
        
        weight_content = np.empty(len(part_of_model), dtype=float)
        for i in range(len(part_of_model)):
            for j in range(len(nama_mua)):
                if part_of_model["nama_MUA"][i] == nama_mua[j]:
                    weight_content[i] = value_mua[j]
        
        final_res = part_of_model.copy()
        final_res = final_res[['nama', 'nama_MUA']]
        final_res['user'] = weight_user
        final_res['content'] = weight_content
        
        mean_weight = np.zeros(len(part_of_model), dtype=float)
        for i in range(len(part_of_model)):
            mean_weight[i] = statistics.harmonic_mean([weight_user[i] + np.min(weight_user[np.nonzero(weight_user)]),  weight_content[i]]) 
        final_res['mean_'] = mean_weight
        higher_value = final_res.copy()
        higher_value = final_res.groupby("nama", as_index=False).mean_.agg(["mean"])
        u = final_res.groupby("nama", as_index=False).user.agg(["mean"])
        higher_value['weight_user'] = u['mean']
        c = final_res.groupby("nama", as_index=False).content.agg(["mean"])
        higher_value['weight_content'] = c['mean']
        higher_value = higher_value.sort_values(by=['mean'], ignore_index=True, ascending=False)
        list_nama = higher_value.nama[:15].tolist()
        pivot = part_of_model.copy()
        pivot['mean_'] = mean_weight
        pivot = pivot[pivot.nama.isin(list_nama)].reset_index()
        pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
        pivot_['weight'] = pivot.groupby("nama_MUA").mean_.agg(['mean'])
        pivot_['predicted'] = pivot_["mean"] * pivot_['weight']
        pivot_ = pivot_.sort_values(by=["predicted"], ascending=False)
        value_rating = np.zeros(len(col_model), dtype=float)
        for i in range(len(pivot_.index)):
            for j in range(len(col_model)):
                if value_rating[j] == 0.0:
                    if col_model[j] == pivot_.index[i]:
                        if pivot_['weight'][i] != 0.0:
                            value_rating[j] = pivot_["mean"][i] * pivot_['weight'][i]
                        else:
                            value_rating[j] = weight_content[i]
                else:
                    continue
        
        pivot_label = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_label = pivot_label.values.tolist()
        col_label = pivot_label.columns.tolist()
        final_label = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_label)):
                if col_model[i] == col_label[j]:
                    final_label[i] = val_label[0][j]
                    break

        tmp_data_recs = {'nama_mua':nama_mua, 'score':value_rating}
        mua_recs = pd.DataFrame(tmp_data_recs).sort_values(by=['score'], ignore_index=True, ascending=False)
        pivot_label = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_label = pivot_label.values.tolist()
        col_label = pivot_label.columns.tolist()
        final_label = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_label)):
                if col_model[i] == col_label[j]:
                    final_label[i] = val_label[0][j]
                    break
        
        for i in range(len(final_label)):
            if final_label[i] == 0.0:
                for j in range(len(pivot_['mean'])):
                    if col_model[i] == pivot_.index.tolist()[j]:
                        final_label[i] = pivot_['mean'].tolist()[j]
        actual_hyb.append(final_label)
        table_hyb.append(pivot_)
        rating_hyb.append(value_rating)

    #     mae = mean_absolute_error(value_rating, final_actual[index_user])
    #     mae_hy.append(mae)
        
    # mean_mae_hy = np.mean(mae_hy)
    return table_hyb, rating_hyb, actual_hyb, col_model

In [599]:
hybrid_based_fil, value_rating_hyb, actual_hyb, col_model = hybrid_evaluation(data_for_model, data_for_user, selected_user, query_user)

In [600]:
hybrid_based_fil[1]

,count,mean,weight,predicted
nama_MUA,,,,
Ayuk Rias Pengantin,3,4.333333,9.314460e-01,4.036266e+00
Larasnada Make Up,5,4.200000,9.569764e-01,4.019301e+00
Laura Make Up & Bridal,5,4.200000,9.559623e-01,4.015042e+00
Yasmin MUA dan Henna Jember,2,4.000000,9.968974e-01,3.987589e+00
Dinda Wijaya Make Up,6,3.666667,9.925762e-01,3.639446e+00
Bonita MUA Jember,4,4.000000,8.961214e-01,3.584486e+00
Ayunia Bridal dan Salon,4,3.500000,9.939544e-01,3.478841e+00
Ulfa Beauty MUA,1,4.000000,8.077131e-01,3.230852e+00
Liyan Make Up Jember,1,4.000000,8.077094e-01,3.230838e+00


In [601]:
col_model

['Abha Beauty Make Up Artist',
 'Aira Salon',
 'Arachma Make Up',
 'Ardie Make Up & Salon',
 'Aulia Rias Pengantin',
 'Ayuk Rias Pengantin',
 'Ayunia Bridal dan Salon',
 'Bonita MUA Jember',
 'Dinda Wijaya Make Up',
 'Eka Make Over',
 'Ella Surya Make Up',
 'Gasebo Make Up Dekorasi',
 'Griya Citra Ayu',
 'Griya Putri Make Up',
 'Hiekmaa MUA',
 'Ifa Salon',
 'Intan Rias Pengantin',
 'Iqlim Rias Pengantin',
 'KL Make Up Private Studio',
 'Khaliza Make Up',
 'Larasnada Make Up',
 'Laura Make Up & Bridal',
 'Liyan Make Up Jember',
 'MUA Niara Puspita',
 'Nonarika Make Up & Hairdo',
 'Novi Make Up',
 'RR Make Up',
 'Rias Manten Mariyam',
 'Rias Ning Lida',
 'Rias Pengantin Ana',
 'Rias Pengantin Azizah',
 'Rias Pengantin Criepno Lestari',
 'Rias Pengantin Sapphire',
 'Rosa Indah Griya Rias Pengantin',
 'Salon Fatah',
 'Salon Indah Sari',
 'Salon Puput Jember',
 'Salon Sriwijaya',
 'Salon Sylvia',
 'Salon Vinell',
 'Sanggar Rias Busana Kartika Budaya',
 'Sanggar Rias Cantik Tina',
 'Sanggar 

## COLLABORATIVE FILTERING

In [602]:
def colab_evaluation(data_for_model, data_for_user, selected_user, query_user):
    actual_colab = []
    table_colab = []
    rating_colab = []
    for index_user in range(len(selected_user)):
        _, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[index_user], query_user[index_user])
        nama_mua, _, part_of_model = value_of_mua(query_data_todict, part_of_model)
        pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        col_model = pivot.columns.tolist()
        val_model = np.array(pivot.values.tolist(), dtype=float)
        nama_user = np.array(pivot.index.tolist())
        pivot_query = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_query = pivot_query.values.tolist()
        col_query = pivot_query.columns.tolist()
        feature = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_query)):
                if col_model[i] == col_query[j]:
                    feature[i] = val_query[0][j]
                    break
        
        cos_sim_user = cosine_similarity([feature], val_model)
        
        final_res = part_of_model.copy()
        final_res = final_res[['nama', 'nama_MUA', 'rating']]
        tmp_cos = np.zeros(len(part_of_model), dtype=float)
        for i in range(len(final_res)):
            for j in range(len(nama_user)):
                if final_res['nama'][i] == nama_user[j]:
                    tmp_cos[i] = cos_sim_user[0][j]
                    break

        final_res['mean_'] = tmp_cos
        higher_value = final_res.copy()
        higher_value = final_res.groupby("nama", as_index=False).mean_.agg(["mean"])
        list_nama = higher_value.nama[:15].tolist()
        pivot = final_res.copy()
        pivot = pivot[pivot.nama.isin(list_nama)].reset_index()
        pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
        pivot_['weight'] = pivot.groupby("nama_MUA").mean_.agg(['mean'])
        pivot_['predicted'] = pivot_["mean"] * pivot_['weight']
        pivot_ = pivot_.sort_values(by=["predicted"], ascending=False)
        value_rating = np.zeros(len(col_model), dtype=float)
        for i in range(len(pivot_.index)):
            for j in range(len(col_model)):
                if value_rating[j] == 0.0:
                    if col_model[j] == pivot_.index[i]:
                        value_rating[j] = pivot_["mean"][i] * pivot_["weight"][i]
                else:
                    continue
        tmp_data_recs = {'nama_mua':nama_mua, 'score':value_rating}
        mua_recs = pd.DataFrame(tmp_data_recs).sort_values(by=['score'], ignore_index=True, ascending=False)
        pivot_label = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_label = pivot_label.values.tolist()
        col_label = pivot_label.columns.tolist()
        final_label = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_label)):
                if col_model[i] == col_label[j]:
                    final_label[i] = val_label[0][j]
                    break
        # mae = mean_absolute_error(value_rating, final_actual[index_user])
        # mae_col.append(mae)

        for i in range(len(final_label)):
            if final_label[i] == 0.0:
                for j in range(len(pivot_['mean'])):
                    if col_model[i] == pivot_.index.tolist()[j]:
                        final_label[i] = pivot_['mean'].tolist()[j]
        actual_colab.append(final_label)
        table_colab.append(pivot_)
        rating_colab.append(value_rating)
        
    # mean_mae_col = np.mean(mae_col)
    return table_colab, rating_colab, actual_colab

In [487]:
# def colab_evaluation(data_for_model, data_for_user, selected_user, query_user):
#     actual_colab = []
#     table_colab = []
#     rating_colab = []
#     for index_user in range(len(selected_user)):
#         _, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[index_user], query_user[index_user])
#         nama_mua, _, part_of_model = value_of_mua(query_data_todict, part_of_model)
#         pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
#         col_model = pivot.columns.tolist()
#         val_model = np.array(pivot.values.tolist(), dtype=float)
#         nama_user = np.array(pivot.index.tolist())
#         pivot_query = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
#         val_query = pivot_query.values.tolist()
#         col_query = pivot_query.columns.tolist()
#         feature = np.zeros(len(col_model), dtype=float)
#         for i in range(len(col_model)):
#             for j in range(len(col_query)):
#                 if col_model[i] == col_query[j]:
#                     feature[i] = val_query[0][j]
#                     break
#         knn = NearestNeighbors(metric='cosine', algorithm='brute')
#         knn.fit(val_model)
#         distances, indices = knn.kneighbors([feature], n_neighbors=10)
#         list_nama = []
#         for ln in indices[0]:
#              list_nama.append(nama_user[ln])
#         pivot = data_for_model[data_for_model.nama.isin(list_nama)].reset_index()
#         tmp_cos = np.empty(len(pivot), dtype=float)
#         for i in range(len(pivot.nama)):
#             for j in range(len(list_nama)):
#                 if pivot.nama[i] == list_nama[j]:
#                     tmp_cos[i] = distances[0][j]
#         pivot['cos'] = tmp_cos
#         pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
#         pivot_['cos'] = pivot.groupby("nama_MUA").cos.agg(['mean'])
#         pivot_['predicted'] = pivot_["mean"] * pivot_["cos"]
#         pivot_ = pivot_.sort_values(by=['predicted'], ascending=False)
#         value_rating = np.zeros(len(col_model), dtype=float)
#         for i in range(len(pivot_.index)):
#             for j in range(len(col_model)):
#                 if value_rating[j] == 0.0:
#                     if col_model[j] == pivot_.index[i]:
#                         value_rating[j] = pivot_["mean"][i] * pivot_["cos"][i]
#                 else:
#                     continue
#         tmp_data_recs = {'nama_mua':nama_mua, 'score':value_rating}
#         mua_recs = pd.DataFrame(tmp_data_recs).sort_values(by=['score'], ignore_index=True, ascending=False)
#         pivot_label = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
#         val_label = pivot_label.values.tolist()
#         col_label = pivot_label.columns.tolist()
#         final_label = np.zeros(len(col_model), dtype=float)
#         for i in range(len(col_model)):
#             for j in range(len(col_label)):
#                 if col_model[i] == col_label[j]:
#                     final_label[i] = val_label[0][j]
#                     break
#         # mae = mean_absolute_error(value_rating, final_actual[index_user])
#         # mae_col.append(mae)

#         for i in range(len(final_label)):
#             if final_label[i] == 0.0:
#                 for j in range(len(pivot_['mean'])):
#                     if col_model[i] == pivot_.index.tolist()[j]:
#                         final_label[i] = pivot_['mean'].tolist()[j]
#         actual_colab.append(final_label)
#         table_colab.append(pivot_)
#         rating_colab.append(value_rating)
        
#     # mean_mae_col = np.mean(mae_col)
#     return table_colab, rating_colab, actual_colab

In [603]:
colab_based_fil, value_rating_colab, actual_colab = colab_evaluation(data_for_model, data_for_user, selected_user, query_user)

In [604]:
# colab_based_fil[colab_based_fil.score > 0.0]

In [605]:
colab_based_fil[0]

,count,mean,weight,predicted
nama_MUA,,,,
Bonita MUA Jember,2,4.000000,1.000000,4.000000
Larasnada Make Up,3,3.666667,0.735653,2.697393
Ardie Make Up & Salon,4,3.500000,0.593704,2.077965
Laura Make Up & Bridal,1,5.000000,0.206958,1.034790
Eka Make Over,3,5.000000,0.187409,0.937043
Rias Manten Mariyam,3,5.000000,0.187409,0.937043
Liyan Make Up Jember,1,4.000000,0.206958,0.827832
Ulfa Beauty MUA,1,4.000000,0.206958,0.827832
Rias Pengantin Criepno Lestari,3,3.000000,0.187409,0.562226


## CONTENT BASED FILTERING

In [606]:
def content_evaluation(data_for_model, data_for_user, selected_user, query_use):
    actual_content = []
    table_content = []
    rating_content = []
    for index_user in range(len(selected_user)):
        _, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[index_user], query_user[index_user])
        nama_mua, value_mua, part_of_model = value_of_mua(query_data_todict, part_of_model)

        weight_content = np.empty(len(part_of_model), dtype=float)
        for i in range(len(part_of_model)):
            for j in range(len(nama_mua)):
                if part_of_model["nama_MUA"][i] == nama_mua[j]:
                    weight_content[i] = value_mua[j]

        pm = part_of_model.copy()
        pm["cos"] = weight_content
        pivot_ = pm.groupby("nama_MUA").rating.agg(['count','mean'])
        pivot_['cos'] = pm.groupby("nama_MUA").cos.agg(['mean'])
        pivot_['predicted'] = pivot_["mean"] * pivot_["cos"]
        pivot_ = pivot_.sort_values(by=['predicted'], ascending=False)
        pivot_ = pivot_[:15]
        value_rating = np.zeros(len(col_model), dtype=float)
        for i in range(len(pivot_.index)):
            for j in range(len(col_model)):
                if value_rating[j] == 0.0:
                    if col_model[j] == pivot_.index[i]:
                        value_rating[j] = pivot_["mean"][i] * pivot_["cos"][i]
                else:
                    continue
        tmp_data_recs = {'nama_mua':nama_mua, 'score':value_rating}
        mua_recs = pd.DataFrame(tmp_data_recs).sort_values(by=['score'], ignore_index=True, ascending=False)
        pivot_label = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_label = pivot_label.values.tolist()
        col_label = pivot_label.columns.tolist()
        final_label = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_label)):
                if col_model[i] == col_label[j]:
                    final_label[i] = val_label[0][j]
                    break
        # mae = mean_absolute_error(value_rating, final_actual[index_user])
        # mae_con.append(mae)

        for i in range(len(final_label)):
            if final_label[i] == 0.0:
                for j in range(len(pivot_['mean'])):
                    if col_model[i] == pivot_.index.tolist()[j]:
                        final_label[i] = pivot_['mean'].tolist()[j]
        actual_content.append(final_label)
        table_content.append(pivot_)
        rating_content.append(value_rating)
        
    # mean_mae_con = np.mean(mae_con)
    return table_content, rating_content, actual_content

In [607]:
content_based_fil, value_rating_content, actual_content = content_evaluation(data_for_model, data_for_user, selected_user, query_user)

In [608]:
# content_based_fil[content_based_fil.score > 0.0]

In [609]:
content_based_fil[0]

,count,mean,cos,predicted
nama_MUA,,,,
Sanggar Rias Nining,5,5.000000,0.999981,4.999903
Abha Beauty Make Up Artist,4,5.000000,0.999981,4.999903
RR Make Up,3,5.000000,0.999981,4.999903
Khaliza Make Up,3,5.000000,0.999980,4.999899
Sanggar Rias Cantik Tina,2,5.000000,0.999967,4.999833
Wedding Gallery Make Over,2,5.000000,0.999967,4.999833
Winda Wijaya MUA & Beauty Studio,3,5.000000,0.999967,4.999833
Aira Salon,2,5.000000,0.999967,4.999833
Aulia Rias Pengantin,4,5.000000,0.999966,4.999828


In [610]:
merge_actual = [[i, j, z] for i,j,z in zip(actual_colab, actual_content, actual_hyb)]
final_actual = []
for i in range(len(merge_actual)):
    tmp = []
    for j in range(len(merge_actual[i][0])):
        sfmean = statistics.fmean([merge_actual[i][0][j], merge_actual[i][1][j], merge_actual[i][2][j]])
        tmp.append(round(sfmean))
    final_actual.append(tmp)
    

In [632]:
def mae_evaluation(predicted_data, actual_data, hybrid=False):
    maeUser = []
    rmseUser = []

    for i in range(len(predicted_data)):
        mae = mean_absolute_error(predicted_data[i], actual_data[i])
        rmse = mean_squared_error(predicted_data[i], actual_data[i], squared=False)
        maeUser.append(mae)
        rmseUser.append(rmse)
    mae = np.mean(maeUser)
    rmse = np.mean(rmseUser)
    if hybrid == True:
        return mae, rmse, maeUser
    else:
        return mae, rmse

In [633]:
mae_col, rmse_col = mae_evaluation(value_rating_colab, final_actual)
mae_con, rmse_con = mae_evaluation(value_rating_content, final_actual)
mae_hyb, rmse_hyb, mae = mae_evaluation(value_rating_hyb, final_actual, hybrid=True)

In [634]:
data_evaluasi = {
    'Metode' : ['Collaborative Filtering', 'Content Based Filtering', 'Hybrid Filtering'],
    'MAE' : [mae_col, mae_con, mae_hyb],
    'RMSE' : [rmse_col, rmse_con, rmse_hyb]
}

tabel_evaluasi = pd.DataFrame(data_evaluasi)
tabel_evaluasi = tabel_evaluasi.sort_values(by=['MAE'], ignore_index=True)
tabel_evaluasi

,Metode,MAE,RMSE
0,Hybrid Filtering,1.129168,1.515565
1,Content Based Filtering,1.533222,1.984384
2,Collaborative Filtering,1.707600,2.197564


In [635]:
def testing_user(data_for_model, data_for_user, index_usr):
    query_data_todict, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[index_usr], query_user[index_usr])
    nama_mua, value_mua, part_of_model = value_of_mua(query_data_todict, part_of_model)
    pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
    col_model = pivot.columns.tolist()
    val_model = np.array(pivot.values.tolist(), dtype=float)
    nama_user = np.array(pivot.index.tolist())
    pivot_query = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
    val_query = pivot_query.values.tolist()
    col_query = pivot_query.columns.tolist()
    feature = np.zeros(len(col_model), dtype=float)
    for i in range(len(col_model)):
        for j in range(len(col_query)):
            if col_model[i] == col_query[j]:
                feature[i] = val_query[0][j]
                break
    
    nmf_model = NMF(3)
    f = nmf_model.fit_transform([feature])
    m = nmf_model.fit_transform(val_model)
    cos_sim_user = cosine_similarity([feature], val_model)

    weight_user = np.empty(len(part_of_model), dtype=float)
    for i in range(len(part_of_model)):
        for j in range(len(nama_user)):
            if part_of_model["nama"][i] == nama_user[j]:
                weight_user[i] = cos_sim_user[0][j]

    weight_user = np.empty(len(part_of_model), dtype=float)
    for i in range(len(part_of_model)):
        for j in range(len(nama_user)):
            if part_of_model["nama"][i] == nama_user[j]:
                weight_user[i] = cos_sim_user[0][j]
        
    weight_content = np.empty(len(part_of_model), dtype=float)
    for i in range(len(part_of_model)):
        for j in range(len(nama_mua)):
            if part_of_model["nama_MUA"][i] == nama_mua[j]:
                weight_content[i] = value_mua[j]
        
    final_res = part_of_model.copy()
    final_res = final_res[['nama', 'nama_MUA']]
    final_res['user'] = weight_user
    final_res['content'] = weight_content

    mean_weight = np.zeros(len(part_of_model), dtype=float)
    for i in range(len(part_of_model)):
        mean_weight[i] = statistics.harmonic_mean([weight_user[i] + np.min(weight_user[np.nonzero(weight_user)]),  weight_content[i]]) 
    final_res['mean_'] = mean_weight
    higher_value = final_res.copy()
    higher_value = final_res.groupby("nama", as_index=False).mean_.agg(["mean"])
    u = final_res.groupby("nama", as_index=False).user.agg(["mean"])
    higher_value['weight_user'] = u['mean']
    c = final_res.groupby("nama", as_index=False).content.agg(["mean"])
    higher_value['weight_content'] = c['mean']
    higher_value = higher_value.sort_values(by=['mean'], ignore_index=True, ascending=False)
    print(higher_value)
    list_nama = higher_value.nama[:15].tolist()
    pivot = part_of_model.copy()
    pivot['mean_'] = mean_weight
    pivot = pivot[pivot.nama.isin(list_nama)].reset_index()
    distance = pivot[['nama_MUA','distance']]
    produk = pivot[['nama_MUA', 'sim']]
    desc = pivot[['nama_MUA', 'produk_makeup', 'shade', 'skin_color', 'skin_undertone']]
    desc['desc'] = desc['produk_makeup'] + ' - ' + desc['shade'] + ' - ' + desc['skin_color'] + ' - ' + desc['skin_undertone']
    pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
    pivot_['nama_MUA'] = desc.groupby('nama_MUA').first().index.to_list()
    pivot_['weight'] = pivot.groupby("nama_MUA").mean_.agg(['mean'])
    pivot_['score'] = pivot_["mean"] * pivot_['weight']
    pivot_['desc'] = desc.groupby('nama_MUA').agg({'desc': lambda x: list(x)}).reset_index()['desc'].values
    pivot_['distance'] = distance.groupby("nama_MUA").distance.agg(['mean'])
    pivot_['sim'] = produk.groupby("nama_MUA").sim.agg(['mean'])
    pivot_ = pivot_.sort_values(by=['score'], ignore_index=True, ascending=False)
    return pivot_

In [636]:
#highest MAE
mae[mae.index(min(mae))]

0.9189890630755427

In [637]:
#highest
idx_high = mae.index(min(mae))
data_for_model.iloc[[selected_user[idx_high]]]

,no,nama,nama_MUA,latitude,longitude,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating,desc
101,61,maulina vira,Nonarika Make Up & Hairdo,-8.169978,113.728703,la tulipe cover foundation,suntan (sawo matang),medium,warm,0,-8.26545,113.530825,2,la tulipe cover foundation suntan sawo matang ...


In [638]:
h_pivot = testing_user(data_for_model, data_for_user, idx_high)
h_pivot

                              nama      mean  weight_user  weight_content
0                     maulina vira  1.078562     1.000000        0.999971
1           zainata hanum masruroh  0.797991     0.493333        0.999969
2                    anissa serina  0.772708     0.459055        0.999969
3         vika aprilia sinta putri  0.608425     0.266667        0.999967
4                 anggia pradita a  0.557822     0.216236        0.999964
5                        dwi rahma  0.508707     0.170561        0.999976
6                        adera zia  0.508706     0.170561        0.999970
7            arsya putri rifasanti  0.291416     0.000000        0.999981
8              dillah ayu firnanda  0.291416     0.000000        0.999981
9                    anggun yeriam  0.291416     0.000000        0.999981
10                 indah febrianti  0.291416     0.000000        0.999981
11             siti aisyah humaira  0.291416     0.000000        0.999981
12                    novia wijaya  0.

,count,mean,nama_MUA,weight,score,desc,distance,sim
0,3,4.666667,Ulfa Beauty MUA,0.976613,4.557528,[la tulipe cover foundation - natural (putih) ...,25.68,0.039483
1,3,5.000000,Winda Wijaya MUA & Beauty Studio,0.891512,4.457561,[la tulipe cover foundation - suntan (sawo mat...,22.21,0.034852
2,2,5.000000,Wedding Gallery Make Over,0.797989,3.989945,[la tulipe cover foundation - natural (putih) ...,18.79,0.044417
3,1,5.000000,Laura Make Up & Bridal,0.772708,3.863541,[la tulipe cover foundation - natural (putih) ...,16.25,0.030268
4,1,4.000000,Liyan Make Up Jember,0.772706,3.090825,[la tulipe cover foundation - natural (putih) ...,1047.88,0.050044
5,2,5.000000,Sanggar Rias Cantik Tina,0.608425,3.042125,[la tulipe cover foundation - rachel 1 (kuning...,16.07,0.031331
6,6,4.333333,Griya Putri Make Up,0.697293,3.021603,[la tulipe cover foundation - suntan (sawo mat...,21.03,0.034171
7,2,5.000000,Aulia Rias Pengantin,0.557822,2.789109,[la tulipe cover foundation - suntan (sawo mat...,23.28,0.044381
8,7,3.857143,Bonita MUA Jember,0.672768,2.594962,[LT Pro smooth corrector cream foundation - na...,22.04,0.049052
9,1,5.000000,Ayuk Rias Pengantin,0.508707,2.543536,[Wardah Everyday Luminous Liquid Foundation - ...,5.25,0.024393


In [627]:
h_pivot.to_excel('highest2.xlsx')

In [639]:
#lowest MAE
mae[mae.index(max(mae))]

1.3617685432936049

In [640]:
#lowest
idx_low = mae.index(max(mae))
data_for_model.iloc[[selected_user[idx_low]]]

,no,nama,nama_MUA,latitude,longitude,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating,desc
45,43,callista febriana erwanto,Ardie Make Up & Salon,-8.144589,113.719387,la tulipe cover foundation,suntan (sawo matang),medium,warm,0,-8.157255,113.652074,4,la tulipe cover foundation suntan sawo matang ...


In [641]:
h_pivot = testing_user(data_for_model, data_for_user, idx_low)
h_pivot

                              nama      mean  weight_user  weight_content
0        callista febriana erwanto  1.107521     1.000000        0.999980
1                    eka febrianti  0.880501     0.545545        0.999981
2               rodhiyyatul hikmah  0.880499     0.545545        0.999975
3       yuricha mahargiyani wijaya  0.675129     0.268608        0.999969
4              calista devi aqilah  0.650433     0.240981        0.999975
5            arsya putri rifasanti  0.388371     0.000000        0.999981
6            sonia arista firnanda  0.388371     0.000000        0.999981
7           azahra aura wiritanaya  0.388371     0.000000        0.999981
8                    anggun yeriam  0.388371     0.000000        0.999981
9              dillah ayu firnanda  0.388371     0.000000        0.999981
10                 indah febrianti  0.388371     0.000000        0.999981
11             siti aisyah humaira  0.388371     0.000000        0.999981
12                    novia wijaya  0.

,count,mean,nama_MUA,weight,score,desc,distance,sim
0,5,4.600000,Rias Manten Mariyam,1.016714,4.676883,[la tulipe cover foundation - rachel 1 (kuning...,1.91,0.025146
1,5,4.600000,Eka Make Over,0.934566,4.299003,[la tulipe cover foundation - natural (putih) ...,54.12,0.028436
2,7,4.571429,Salon Fatah,0.846762,3.870913,[la tulipe cover foundation - suntan (sawo mat...,33.67,0.062789
3,2,5.000000,Ayunia Bridal dan Salon,0.675128,3.375640,[Wardah Lightening Liquid Foundation 25ml - ne...,6.22,0.021168
4,3,3.000000,Rias Pengantin Criepno Lestari,1.107521,3.322564,[la tulipe cover foundation - suntan (sawo mat...,28.71,0.033676
5,3,5.000000,Khaliza Make Up,0.650434,3.252171,[la tulipe cover foundation - natural (putih) ...,23.39,0.038416
6,3,5.000000,Yulia Salon & Bridal,0.650431,3.252154,[la tulipe cover foundation - natural (putih) ...,25.48,0.024942
7,2,4.000000,Laura Make Up & Bridal,0.675130,2.700518,[la tulipe cover foundation - suntan (sawo mat...,5.34,0.065779
8,3,4.000000,Novi Make Up,0.650434,2.601736,[LT Pro smooth corrector cream foundation - na...,28.29,0.038035
9,2,3.000000,Dinda Wijaya Make Up,0.675127,2.025381,[la tulipe cover foundation - suntan (sawo mat...,6.70,0.128181


In [631]:
h_pivot.to_excel('lowest2.xlsx')

In [479]:
# rating_test = []
# for i in range(len(query_user)):
#     rating_dict = dict()
#     for j in range(len(col_model)):
#         rating_dict[col_model[j]] = final_actual[i][j]
#     rating_test.append(rating_dict)

# dataTesting = pd.DataFrame.from_dict(rating_test)
# dataTesting['user'] = query_user
# dataTesting.set_index(['user'])
# dataTesting.to_excel('dataTesting2.xlsx')

In [43]:
# predicted_rating = []
# for i in range(len(query_user)):
#     rating_dict = dict()
#     for j in range(len(col_model)):
#         rating_dict[col_model[j]] = value_rating_hyb[i][j]
#     predicted_rating.append(rating_dict)

# predictedRating = pd.DataFrame.from_dict(predicted_rating)
# predictedRating['user'] = query_user
# predictedRating.set_index(['user'])
# predictedRating.to_excel('predictedRating2.xlsx')